# ibes_ltg

> Retrieve and process data on EPS Long-Term Growth (LTG) forecasts from WRDS IBES database.

***

The Forecast Period End Date (`fpedats`) is the ending month and year of the fiscal period to which the estimate applies (unless you're asking for a long-term-growth estimate, in which case the horizon is 3-5 years, so no explicity fiscal period is applicable). 

The Activation Date (`actdats`) is the date that the forecast/actual was recorded by Thomson Reuters. 

The Announce Date (`anndats`) is the date that the forecast/actual was reported. 

The Review Date (`revdats`) is most recent date that an estimate was confirmed as accurate. 

The Forecast Period Indicator (`fpi`) contains information about the horizon (how far into the future we are estimating). Key values: '0' for LTG, '1'-'5' for 1 to 5 years in the future, '6'-'9' for 1 to 4 quarters in the future. Farther horizons are available but they are extremely poorly populated.

Note that for Long Term Growth (LTG) estimates (`fpi='0'`), you must NOT select "Forecast Period End Date" as the Date Variable or the query will not return any estimates.

`TICKER` is the IBES ticker, which is not necessarily the same as the offical ticker of the firm.

***
It is possible for a contributing broker to provide multiple revisions to an estimate on the same day. In
this scenario, all estimates are available in the Detail history files and only the most current estimate is
included in the mean.

The Brokers (`estimator`) and Analysts (`analys`) are provided under numeric codes. 

***
**Estimate Revisions**

There are estimates which are dated “after” the announcement date. We
have no explanation other than the entry is in error.

Announcement of earnings will increment the FPI variable by 1 in all IBES
records for which review date (REVDATS)> report date (ANNDATS_ACT)

If at the time of the next review date the analyst at the same brokerage changes
her forecast for the same (TICKER, ANNDATS, FPEDATS, FPI, MEASURE,
USFIRM) combination, IBES will add a new observation. If the forecast
remains unchanged, IBES will not add new observations, but will adjust the
review date accordingly (REVDATS)

***

In [ ]:
#| default_exp wrds.ibes_ltg

In [ ]:
#|exports
from __future__ import annotations
from typing import List

import pandas as pd

import pandasmore as pdm
from finsets.wrds import wrds_api

In [ ]:
#| exports
PROVIDER = 'Refinitiv via WRDS'
URL = 'https://wrds-www.wharton.upenn.edu/pages/get-data/ibes-thomson-reuters/ibes-academic/unadjusted-detail/history/'
LIBRARY = 'ibes'
TABLE = 'detu_epsus'
LINK_LIBRARY = 'wrdsapps_link_crsp_ibes'
LINK_TABLE = 'ibcrsphist'
FREQ = 'M'
MIN_YEAR = 1925
MAX_YEAR = None
ENTITY_ID_IN_RAW_DSET = 'permno'
ENTITY_ID_IN_CLEAN_DSET = 'permno'
TIME_VAR_IN_RAW_DSET = 'date'
TIME_VAR_IN_CLEAN_DSET = f'{FREQ}date'

In [ ]:
#| export
def list_all_vars() -> pd.DataFrame:
    "Collects names of all available variables from WRDS f`{LIBRARY}.{TABLE}`"

    try:
        db = wrds_api.Connection()
        funda = db.describe_table(LIBRARY,TABLE).assign(wrds_library=LIBRARY, wrds_table=TABLE)
    finally:
        db.close()

    return funda[['name','type','wrds_library','wrds_table']]

In [ ]:
#| eval: false
all_vars = list_all_vars()

Loading library list...
Done
Approximately 34393864 rows in ibes.detu_epsus.


In [ ]:
#| eval: false
all_vars.name.count()

np.int64(21)

In [ ]:
#| export 
def default_raw_vars():
    return ['ticker', 'value', 'fpi', 'anndats', 'fpedats', 'revdats', 'actdats', 'estimator', 'analys', 'pdf']

In [ ]:
#| export
def parse_varlist(vars: List[str]|str=None, #list of variables requested by user
                  required_vars: List[str] = [], #list of variables that will get downloaded, even if not in `vars`
                  prefix: str='a.', #string to add in front of each variable name when we build the SQL string of variable names
                  ) -> str:
    """Adds required variables to requested variables, validates them, and builds the SQL string with their names"""

    if vars=='*': return f'{prefix}*' 

    # Build full list of variables that will be downloaded
    if vars is None: vars = default_raw_vars()
    if required_vars is None: req_vars = []
    vars =  required_vars + [x for x in vars if x not in required_vars] #in case `vars` already contains some of the required variables

    # Validate variables to be downloaded (make sure that they are in the target database)
    valid_vars = list(list_all_vars().name)
    invalid_vars = [v for v in vars if v not in valid_vars]
    if invalid_vars: raise ValueError(f"These vars are not in the database: {invalid_vars}") 

    return ','.join([f'{prefix}{var_name}' for var_name in vars])

In [ ]:
#| eval: false
parse_varlist(['value','fpi'])

Loading library list...
Done
Approximately 34393864 rows in ibes.detu_epsus.


'a.value,a.fpi'

In [ ]:
#| export
def get_raw_data(vars: List[str]=None, # If None, downloads `default_raw_vars`; `permno`, `ticker`, and `anndats` added by default
            required_vars: List[str] = ['ticker', 'anndats'], #list of variables that will get downloaded, even if not in `vars`
             nrows: int=None, #Number of rows to download. If None, full dataset will be downloaded
             start_date: str=None, # Start date in MM/DD/YYYY format
             end_date: str=None, #End date in MM/DD/YYYY format; if None, defaults to current date
             permno_match_score: tuple=(1,), #accuracy of permno-ibes link. 1-6. 1 is best. use >1 with caution.
             ) -> pd.DataFrame:
    """Downloads `vars` from `start_date` to `end_date` from WRDS `ibes.detu_epsus` library and adds PERMNO from CRSP"""

    wrds_api.validate_dates([start_date, end_date])
    vars = parse_varlist(vars, prefix='a.', required_vars=required_vars)

    sql_string=f"""SELECT {vars}, b.permno
                        FROM {LIBRARY}.{TABLE} AS a
                        LEFT JOIN {LINK_LIBRARY}.{LINK_TABLE} AS b
                        ON a.ticker = b.ticker
                        WHERE a.anndats BETWEEN b.sdate AND b.edate
                                AND fpi='0'
                """
    if permno_match_score is not None: sql_string += r" AND score IN %(permno_match_score)s"
    if start_date is not None: sql_string += r" AND anndats >= %(start_date)s"
    if end_date is not None: sql_string += r" AND anndats <= %(end_date)s"
    if nrows is not None: sql_string += r" LIMIT %(nrows)s"

    return wrds_api.download(sql_string,
                             params={'permno_match_score': permno_match_score,
                                 'start_date':start_date, 'end_date':end_date, 'nrows':nrows})

In [ ]:
#| eval: false
ltg = get_raw_data(start_date='01/01/2019', end_date='01/01/2022', nrows=1)
ltg.head()

Loading library list...
Done
Approximately 34393864 rows in ibes.detu_epsus.
Loading library list...
Done


,ticker,anndats,value,fpi,fpedats,revdats,actdats,estimator,analys,pdf,permno
0,0001,2019-01-04,-174.4,0,<NA>,2019-01-04,2019-01-04,183.0,48368.0,D,14392


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()